# Task 1. Password Uniqueness Check Using Bloom Filter

Create a function to check password uniqueness using a Bloom filter. This function should determine whether a password has been used before without the need to store the passwords themselves.

## Implementation

In [1]:
"""Password uniqueness check using Bloom filter implementation.

This notebook provides:
  - A Bloom filter implementation for password uniqueness checking.
  - Performance testing for the Bloom filter.
    - A HyperLogLog implementation for estimating unique IP addresses in log files.
    - Reading and processing large log files in chunks using parallel processing.

Follows Google Python Style Guide:
  https://google.github.io/styleguide/pyguide.html
"""

# Standard library imports
import secrets
import string
from typing import Dict, List, Set
import time
import math

# Third-party imports
import mmh3

In [2]:
class BloomFilter:
    """A probabilistic data structure for fast membership queries.
    
    A Bloom filter is a space-efficient probabilistic data structure that is
    used to test whether an element is a member of a set. False positive matches
    are possible, but false negatives are not.
    """

    def __init__(self, size: int, num_hashes: int) -> None:
        """Initialize the Bloom filter.
        
        Args:
            size: The size of the bit array.
            num_hashes: The number of hash functions to use.
        """
        self.size: int = size
        self.num_hashes: int = num_hashes
        self.bit_array: List[int] = [0] * size

    def _get_hash_indices(self, item: str) -> List[int]:
        """Generate hash indices for an item.
        
        Args:
            item: The item to hash.
            
        Returns:
            List of hash indices.
        """
        return [mmh3.hash(item, i) % self.size for i in range(self.num_hashes)]

    def add(self, item: str) -> None:
        """Add an item to the Bloom filter.
        
        Args:
            item: The item to add.
        """
        normalized_item: str = "" if item is None else str(item)
        for index in self._get_hash_indices(normalized_item):
            self.bit_array[index] = 1

    def contains(self, item: str) -> bool:
        """Check if an item might be in the set.
        
        Args:
            item: The item to check.
            
        Returns:
            True if the item might be in the set, False if it's definitely not.
        """
        return all(self.bit_array[index] == 1 
                  for index in self._get_hash_indices(item))

In [3]:
def generate_strong_password(length: int = 16) -> str:
    """Generate a cryptographically secure password.
    
    The password will contain at least one character from each category:
    lowercase, uppercase, digits, and punctuation.
    
    Args:
        length: The desired password length. Must be at least 4.
        
    Returns:
        A randomly generated password.
        
    Raises:
        ValueError: If length is less than 4.
    """
    if length < 4:
        raise ValueError("Password length must be at least 4")
    
    # Ensure at least one character from each required category
    required_chars: list[str] = [
        secrets.choice(string.ascii_lowercase),
        secrets.choice(string.ascii_uppercase),
        secrets.choice(string.digits),
        secrets.choice(string.punctuation),
    ]
    
    # Fill remaining positions with random characters from all categories
    char_pool: str = string.ascii_letters + string.digits + string.punctuation
    remaining_chars: list[str] = [
        secrets.choice(char_pool) for _ in range(length - len(required_chars))
    ]
    
    # Combine and shuffle all characters
    all_chars: list[str] = required_chars + remaining_chars
    secrets.SystemRandom().shuffle(all_chars)
    
    return ''.join(all_chars)


def generate_passwords(count: int = 1, length: int = 16) -> List[str]:
    """Generate multiple unique passwords.
    
    Args:
        count: Number of passwords to generate.
        length: Length of each password.
        
    Returns:
        List of unique passwords.
    """
    passwords: list[str] = []
    seen: Set = set()
    
    # Generate unique passwords
    while len(passwords) < count:
        password: str = generate_strong_password(length)
        if password not in seen:
            seen.add(password)
            passwords.append(password)
    
    return passwords


def check_password_uniqueness(
    bloom_filter: BloomFilter, passwords: List[str]
) -> Dict[str, str]:
    """Check password uniqueness using a Bloom filter.
    
    Args:
        bloom_filter: The Bloom filter containing previously used passwords.
        passwords: List of passwords to check.
        
    Returns:
        Dictionary mapping passwords to their status ('Already used' or 'Available').
    """

    return {
        password: "Already used" if bloom_filter.contains(password) else "Available"
        for password in passwords
    }


def test_performance(pass_count: int = 100_000, false_positive_prob: float = 0.01) -> None:
    """Tests and shows performance with large datasets and calculates the false-positive rate.

    Args:
        pass_count: Number of passwords to generate and check.
        false_positive_prob: Desired false-positive probability.
    """
    print("\n\nTest performance with large datasets:")

    # Calculating optimal size and number of hash functions

    size: int = int(-(pass_count * math.log(false_positive_prob)) / (math.log(2) ** 2))
    num_hashes: int = int((size / pass_count) * math.log(2))

    # Initialize Bloom filter
    bloom_filter: BloomFilter = BloomFilter(size=size, num_hashes=num_hashes)

    print(f"Testing with {pass_count:,} passwords.")
    print(f"Desired false positive rate: {false_positive_prob:.2%}")
    print(f"Optimal filter size: {size:,} bits")
    print(f"Optimal number of hashes: {num_hashes}")

    # Adding passwords to the Bloom filter
    existing_passwords: List[str] = generate_passwords(count=pass_count, length=16)

    start_time: float = time.time()
    for password in existing_passwords:
        bloom_filter.add(password)
    addition_time: float = time.time() - start_time
    print(f"Adding {len(existing_passwords):,} passwords to the filter: {addition_time:.4f} sec")

    # Checking for false positives
    # Generate new passwords that are guaranteed not to be in the filter
    new_passwords_to_check: List[str] = generate_passwords(count=pass_count, length=16)

    false_positives: int = 0
    start_time: float = time.time()
    for password in new_passwords_to_check:
        if bloom_filter.contains(password):
            false_positives += 1
    check_time: float = time.time() - start_time

    print(f"Checking {len(new_passwords_to_check):,} new unique passwords: {check_time:.4f} sec")

    false_positive_rate: float = false_positives / pass_count

    print(f"\n\nPerformance Results:")
    print(f"False positives found: {false_positives} out of {pass_count}")
    print(f"Actual false positive rate: {false_positive_rate:.2%}")
    print(f"Memory usage of the filter: {len(bloom_filter.bit_array):,} bits")

## Results

In [4]:
def main() -> None:
    """Demonstrate password uniqueness checking with Bloom filter."""
    # Initialize Bloom filter
    bloom_filter = BloomFilter(size=1000, num_hashes=3)

    # Generate and add existing passwords to the Bloom filter
    PWD_CONT: int = 3
    PWD_NEW: int = 3
    PWD_LEN: int = 12
    existing_passwords: List[str] = generate_passwords(count=PWD_CONT, length=PWD_LEN)
    existing_passwords += [
        None,
        "",
        " ",
        "  ",
        "Ω≤≥∑∏π",
        "123456",
        "!@#$%^&*()",
        "password123",
        "admin123",
        "qwerty123",
    ]

    print("Existing passwords:")
    for password in existing_passwords:
        print(f"  '{password}'")
        bloom_filter.add(password)

    # Generate new passwords to check (some existing + some new)
    new_passwords: List[str] = generate_passwords(count=PWD_NEW, length=PWD_LEN)
    new_passwords += ["password123", "newpassword", "admin123", "guest"]
    passwords_to_check: List[str] = existing_passwords[:PWD_CONT] + new_passwords  # Mix of old and new

    print(f"\nChecking {len(passwords_to_check)} passwords:")

    results: Dict[str, str] = check_password_uniqueness(bloom_filter, passwords_to_check)

    # Display results
    for password, status in results.items():
        print(f"  '{password}' - {status}")

    # Run performance tests
    test_performance(pass_count=10_000_000, false_positive_prob=0.01)


if __name__ == "__main__":
    main()

Existing passwords:
  '>]g+1*y9wPl_'
  '[uc5oh'(r0Qx'
  'YBXI~Z9yij29'
  'None'
  ''
  ' '
  '  '
  'Ω≤≥∑∏π'
  '123456'
  '!@#$%^&*()'
  'password123'
  'admin123'
  'qwerty123'

Checking 10 passwords:
  '>]g+1*y9wPl_' - Already used
  '[uc5oh'(r0Qx' - Already used
  'YBXI~Z9yij29' - Already used
  '$P\`sDx1}0P&' - Available
  '`v2m7VFNs"o(' - Available
  'LO3EI)zY`P@/' - Available
  'password123' - Already used
  'newpassword' - Available
  'admin123' - Already used
  'guest' - Available


Test performance with large datasets:
Testing with 10,000,000 passwords.
Desired false positive rate: 1.00%
Optimal filter size: 95,850,583 bits
Optimal number of hashes: 6
Adding 10,000,000 passwords to the filter: 8.6695 sec
Checking 10,000,000 new unique passwords: 13.7775 sec


Performance Results:
False positives found: 101890 out of 10000000
Actual false positive rate: 1.02%
Memory usage of the filter: 95,850,583 bits
